In [1]:
from PIL import Image
from numpy import asarray
import numpy as np
import params
from CentroidTracker import CentroidTracker, progressBar, showWindows, georeference
import cv2

from model import DeepLabModel, DeepLab

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

In [2]:
def create__label_colormap():
     return np.asarray([
         [0, 0, 0],
         [0, 192, 0],
         [255, 192, 192],
     ])
    
def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
    label: A 2D array with integer type, storing the segmentation label.

    Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

    Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create__label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

In [38]:
ct = CentroidTracker()
current_frame = 0
model = DeepLabModel('model/frozen_inference_graph_cars.pb')

In [45]:
video = str('./input/video/DJI_0008_short.mp4')
output_video = str('./output/video/DJI_0008_short.avi') # Path to Output-Video

In [48]:
cap = cv2.VideoCapture(video)
cap.set(cv2.CAP_PROP_POS_AVI_RATIO,0)

img_array =[]
while True:
    current_frame += 1
    ret, frame = cap.read()
    
    if frame is None:
        print('finished!')
        break
        
        mask = model.run(original_im)
        image_array = np.array(mask)

        # apply threshold
        thresh = threshold_otsu(image_array)
        bw = closing(image_array > thresh, square(3))

        # remove artifacts connected to image border
        cleared = clear_border(bw)
        
        # label image regions
        label_image = label(cleared)
        # to make the background transparent, pass the value of `bg_label`,
        # and leave `bg_color` as `None` and `kind` as `overlay`
        image_label_overlay = label2rgb(label_image, image=image_array, bg_label=0)
        
        #fig, ax = plt.subplots(figsize=(10, 6))
        #ax.imshow(image_label_overlay)
        
        # --- Generate Boundingboxes
        # for region in regionprops(label_image):
        #     # take regions with large enough areas
        #     if region.area >= 100:
        #         # draw rectangle around segmented coins
        #         minr, minc, maxr, maxc = region.bbox
        #         rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
        #                                   fill=False, edgecolor='red', linewidth=2)
        #         ax.add_patch(rect)

        cv2.imshow('Frame', frame)
        cv2.imshow('FG MASK Frame', fgmask)

    keyboard = cv2.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

finished!


In [49]:
# ---- translate array
size = img_array[0].shape[:2][::-1]

IndexError: list index out of range

In [50]:
# ---- save to video
out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

NameError: name 'size' is not defined

In [26]:
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

NameError: name 'img' is not defined

In [23]:
# convert the grayscale image to binary image
ret,thresh = cv2.threshold(gray_image,127,255,0)

NameError: name 'gray_image' is not defined

In [ ]:
# find contours in the binary image
im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
for c in contours:
    # calculate moments for each contour
    M = cv2.moments(c)
    
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    cv2.circle(image_label_overlay, (cX, cY), 5, (255, 255, 255), -1)
    cv2.putText(thresh_img, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    # display the image
    cv2.imshow("Image", thresh_img)
    cv2.waitKey(0)